import packages

In [1]:
import SciNN_G_RNN as sgrnn
import numpy as np
import util
import os
# from IPython.display import clear_output
import copy
import random
import csv
import matplotlib.pyplot as plt
import pngToMp4 as p

set factors

In [2]:
GRAPH_TYPE = 0 # 0 ~ 1
# 0 : 병렬-독립 모형
# 1 : 직렬-종속 모형

ETA_TYPE = 1 # 0 ~ 1
# 0 : 한쪽이 0인 경우
# 1 : 0이 아니되 상대적으로 작은 값을 갖는 경우

TRAINING_SENARIO_NO = 0 # 0 ~ 5 : 맨 처음에는 0으로 시작해야 합니다
# 0 : backprop을 통한 gamma -> 10us f 학습
# 1 : 대조군 평가 : gamma ~> r
# 2 : 붉은색 공포기억 연합화 : r + gamma ~> r
# 3 : 붉은색 공포기억 지우기 : r + gamma ~> gamma ~> r
# 4 : 붉은색 공포기억 덮어씌우기 : r + gamma ~> b + gamma ~> r or b
# 5 : 붉은색 앞으로 가기 학습 : r or r + gamma ~> r

PROJECT_NAME = "2024MO04-2 w heatmap"

DATA_FILE_TO_USE = PROJECT_NAME + \
    f"[{GRAPH_TYPE},{ETA_TYPE},0]"
SAVE_LEARNING_HISTORY_FILE_NAME = PROJECT_NAME + \
    f"[{GRAPH_TYPE},{ETA_TYPE},{TRAINING_SENARIO_NO}]"
SAVE_HEATMAP_FILE_NAME = PROJECT_NAME + \
    f"[{GRAPH_TYPE},{ETA_TYPE},{TRAINING_SENARIO_NO}]"

INPUT_COLOR_DIMENSION = 3  # 변경하면 안됨 (r, g, b)
INPUT_SHOCK_DIMENSION = 1 # (gamma)
INPUT_ENV_DIMENSION = 1
OUTPUT_MOVEMENT_DIMENSION = 2  # 변경하면 안됨 (f, r)
OUPUT_IRRELAVENT_DIMENSION = 1

INTERNAL_A_NEURON_LENGTH = 10
INTERNAL_B_NEURON_LENGTH = 10

BASE_ETA = 0.01
BASE_ETA_BCM = 0.001

TRAIN_EPOCH = 400
TEST_EPOCH = 100
EXPERIMENT_EPOCH = 500

INPUT_ENV_VOLATILITY = 0.01

INITIAL_SPARSITY = 2.27

LIGHT_EXPOSURE_TIME = 20
WAITING_TIME = 10
FREEZE_TIME = 10

SHOCK_EXPOSURE_PROB = 1/2
RED_COLOR_EXPOSURE_PROB = 1/3
BLUE_COLOR_EXPOSURE_PROB = 1/3 # < 1 - RED_COLOR_EXPOSURE_PROB

CORRECT_LOSS_LIMIT = 0.05

SAVE_HEATMAP = True # Heatmap 저장 및 영상 생성 여부, False인 경우 속도 향상됨

In [3]:
# p.pngToMp4(
#     f"output/{SAVE_HEATMAP_FILE_NAME}[{0}]", f"output/{SAVE_HEATMAP_FILE_NAME}[{0}]")

create Graphs

In [4]:
output_length = OUTPUT_MOVEMENT_DIMENSION + OUPUT_IRRELAVENT_DIMENSION
input_ext_length = INPUT_COLOR_DIMENSION + \
    INPUT_SHOCK_DIMENSION + INPUT_ENV_DIMENSION
input_length = input_ext_length + output_length

eta_factor = 0.1 if ETA_TYPE == 1 else 0

graph = []
if GRAPH_TYPE == 0:
    graph = [[input_length, INTERNAL_A_NEURON_LENGTH, INTERNAL_B_NEURON_LENGTH, output_length],
             [[0, 0, 0, 0],
              [BASE_ETA, BASE_ETA, 0, 0],
              [eta_factor*BASE_ETA, 0, eta_factor*BASE_ETA, 0],
              [0, BASE_ETA, eta_factor*BASE_ETA, 0]],
             [[0, 0, 0, 0],
              [eta_factor*BASE_ETA_BCM, eta_factor*BASE_ETA_BCM, 0, 0],
              [BASE_ETA_BCM, 0, BASE_ETA_BCM, 0],
              [0, eta_factor*BASE_ETA_BCM, BASE_ETA_BCM, 0]],
             0,
             3]
else :
    graph = [[input_length, INTERNAL_A_NEURON_LENGTH, INTERNAL_B_NEURON_LENGTH, output_length],
             [[0, 0, 0, 0],
              [BASE_ETA, BASE_ETA, eta_factor*BASE_ETA, 0],
              [0, eta_factor*BASE_ETA, eta_factor*BASE_ETA, 0],
              [0, BASE_ETA, 0, 0]],
             [[0, 0, 0, 0],
              [eta_factor*BASE_ETA_BCM, eta_factor*BASE_ETA_BCM, BASE_ETA_BCM, 0],
              [0, BASE_ETA_BCM, BASE_ETA_BCM, 0],
              [0, eta_factor*BASE_ETA_BCM, 0, 0]],
             0,
             3]

set new G-RNN model

In [5]:


a = sgrnn.SciNN_G_RNN(*graph)
a.setLossFunction("mse")
a.initializeWeight(INITIAL_SPARSITY)
# print(a.W)
# print(a.V_sorted, a.d)
# for i in range(1):
#     a.propagateBP([1,0,0,1,0],[0,1,0])
# a.saveHeatmap("temptemp")

set random input-target generating function

In [6]:
temp_red_prob = RED_COLOR_EXPOSURE_PROB - (1-RED_COLOR_EXPOSURE_PROB-BLUE_COLOR_EXPOSURE_PROB)/25
temp_blue_prob = BLUE_COLOR_EXPOSURE_PROB - (1-RED_COLOR_EXPOSURE_PROB-BLUE_COLOR_EXPOSURE_PROB)/25

def generateColor():
    p = random.random()
    if p < temp_red_prob:
        return {"input_color": [1-random.random()/3, random.random()/3, random.random()/3], "label": "red"}
    elif p < temp_red_prob + temp_blue_prob:
        return {"input_color": [random.random()/3, random.random()/3, 1-random.random()/3], "label": "blue"}
    else:
        input_color = [random.random(), random.random(), random.random()]
        label = ""
        if input_color[0] > 2/3 and input_color[1] < 1/3 and input_color[2] < 1/3:
            label = "red"
        elif input_color[0] < 1/3 and input_color[1] < 1/3 and input_color[2] > 2/3:
            label = "blue"
        else:
            label = "others"
        return {"input_color": input_color, "label": label}

# 0 to 1    
def generateEnv(env_prev):
    return [env_prev[env] + (random.random() - env_prev[env])*INPUT_ENV_VOLATILITY for env in range(INPUT_ENV_DIMENSION)]

create fear reaction

In [7]:
if TRAINING_SENARIO_NO == 0:
    
    if not os.path.isdir(f"output/{SAVE_HEATMAP_FILE_NAME}"):
        os.mkdir(f"output/{SAVE_HEATMAP_FILE_NAME}")

    f = open(f"output/{SAVE_LEARNING_HISTORY_FILE_NAME}.csv",
             "w", encoding="utf-8", newline="")
    wr = csv.writer(f)
    wr.writerow(["no", "iteration", "input_shock", "time", "loss", "success", "correct_count"])
    
    for ep in range(TRAIN_EPOCH + TEST_EPOCH):
        shock_now = random.random() < SHOCK_EXPOSURE_PROB
        correct_count = 0
        shock_stopped = False
        input_color = [random.random() for color in range(INPUT_COLOR_DIMENSION)]
        input_env = [random.random() for env in range(INPUT_ENV_DIMENSION)]
        
        for t in range(LIGHT_EXPOSURE_TIME + WAITING_TIME):
            input_env = generateEnv(input_env)
            input_color = [input_color[color] + 2*random.random()*INPUT_ENV_VOLATILITY -
                         INPUT_ENV_VOLATILITY for color in range(INPUT_COLOR_DIMENSION)]
            if t < LIGHT_EXPOSURE_TIME:
                input_vector = [
                    *input_color, *[1 if shock_now else 0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env]
            else : 
                input_vector = [
                    *input_color, *[0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env]
                
            target_vector = [1 if shock_now else 0, 0 if shock_now else 1] 
            
            if not shock_stopped and ep < TRAIN_EPOCH:
                a.propagateBPConsideringOutput(input_vector, lambda output: [*target_vector, *output[2:]])
            else:
                a.propagateBPConsideringOutput(input_vector, lambda output: output)
                
            if a.loss < CORRECT_LOSS_LIMIT:
                if correct_count > FREEZE_TIME:
                    shock_stopped = True
                else:
                    correct_count += 1
            
            util.printAfterErase(
                f"iteration : {ep+1}; time : {t+1} ; loss : {a.loss} ({a.h[a.o][0:2]}<->{a.target_vector[0:2]})      ")
            wr.writerow([ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,
                        ep+1,
                        shock_now,
                        t+1,
                        a.loss,
                        1 if shock_stopped else 0,
                        correct_count
                        ])
            if SAVE_HEATMAP or (ep+1==TRAIN_EPOCH+TEST_EPOCH):
                a.saveHeatmap(f"output/{SAVE_HEATMAP_FILE_NAME}/{util.fillZero(ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,5)}.png")

    f.close()
    a.saveCurrent(DATA_FILE_TO_USE)
    if SAVE_HEATMAP:
        p.pngToMp4(f"output/{SAVE_HEATMAP_FILE_NAME}",f"output/{SAVE_HEATMAP_FILE_NAME}")
    print("\nDone")
    # clear_output()

In [8]:
# a.loadData(DATA_FILE_TO_USE)
# print(a.W)

connect fear with red light

In [9]:
# color : "red" | "blue" | "random"
def _mseLoss(output, target):
    return sum([(output[i]-target[i])**2 for i in range(len(output))])/len(output)

def trainColorAndShockExperiment(file_index=0, color="red", use_shock=True, reset=False):
    if reset:
        a.loadData(DATA_FILE_TO_USE)
        a.decodeWeightOfBackprop()
        a.saveCurrent("temp")
    if not os.path.isdir(f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]"):
        os.mkdir(f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]")
    
    input_env = [random.random() for env in range(INPUT_ENV_DIMENSION)]
    f = open(f"output/{SAVE_LEARNING_HISTORY_FILE_NAME}[{file_index}].csv",
             "w", encoding="utf-8", newline="")
    wr = csv.writer(f)
    wr.writerow(["no", "iteration", "color", "time",
                "loss", "success", "correct_count", "shock", "freeze", "release"])

    for ep in range(EXPERIMENT_EPOCH):
        input_color_sample = generateColor()
        rand_val = random.random()
        is_correct_color = (input_color_sample["label"] == color) or (color == "random" and (rand_val < 0.5)) 
        shock_now = use_shock and is_correct_color
        correct_count = 0
        target_vector = [1 if is_correct_color else 0, 0 if is_correct_color else 1] 

        for t in range(LIGHT_EXPOSURE_TIME + WAITING_TIME):
            input_env = generateEnv(input_env)
            if t < LIGHT_EXPOSURE_TIME:
                input_vector = [
                    *input_color_sample["input_color"], *[1 if shock_now else 0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env]
            else:
                input_vector = [
                    *[0 for c in range(INPUT_COLOR_DIMENSION)], *[0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env]

            a.propagateBCM(input_vector)

            loss = _mseLoss(a.h[a.o][0:2], target_vector)

            if loss < CORRECT_LOSS_LIMIT:
                if correct_count < FREEZE_TIME:
                    correct_count += 1
            
            util.printAfterErase(
                f"iteration : {ep+1}; time : {t+1} ; loss : {loss}   {rand_val}  ")
            wr.writerow([ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,
                        ep+1,
                        input_color_sample['label'],
                        t+1,
                        loss,
                        correct_count>=FREEZE_TIME,
                        correct_count,
                        shock_now,
                        *a.h[a.o][0:2]
                         ])
            if SAVE_HEATMAP or (ep+1 == EXPERIMENT_EPOCH):
                a.saveHeatmap(
                    f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]/{util.fillZero(ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,5)}.png")

    f.close()
    
    if SAVE_HEATMAP:
        p.pngToMp4(f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]",f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]")
    
    print("\nDone")
    # clear_output()

In [10]:

if TRAINING_SENARIO_NO == 1:
    trainColorAndShockExperiment(0, "random", True, True)
    trainColorAndShockExperiment(1, "red", False)
if TRAINING_SENARIO_NO == 2:
    trainColorAndShockExperiment(0,"red",True, True)
    a.saveCurrent("temp2")
    trainColorAndShockExperiment(1, "red",False)
    a.saveCurrent("temp3")
if TRAINING_SENARIO_NO == 3:
    trainColorAndShockExperiment(0, "red", True, True)
    trainColorAndShockExperiment(1, "random", True)
    trainColorAndShockExperiment(2, "red", False)
if TRAINING_SENARIO_NO == 4:
    trainColorAndShockExperiment(0, "red", True, True)
    trainColorAndShockExperiment(1, "blue", True)
    trainColorAndShockExperiment(2, "red", False)

iteration : 1; time : 1 ; loss : 0.09952532973775666   0.3323119482613578  

iteration : 500; time : 30 ; loss : 0.12066505017404115   0.3912675913577436     
Done
iteration : 500; time : 30 ; loss : 0.12746852096117958   0.7726860848986536      
Done
iteration : 500; time : 30 ; loss : 0.24915569446433472   0.7383762148566005     
Done


activate model

In [11]:
if False:

    input_color = []
    input_env = [random.random() for env in range(INPUT_ENV_DIMENSION)]
    # t-L-1 ~ t-1 범위의 과거 output 저장
    output_vector_list_prev = [[0 for i in range(output_length)] for l in range(a.layer_depth+1)]
    f = open(f"output/{SAVE_LEARNING_HISTORY_FILE_NAME}.csv","w",encoding="utf-8",newline="")
    wr = csv.writer(f)
    wr.writerow(["no", "iteration", "input_color", "input_env", "time", "loss", "success", "correct_count"])

    for ep in range(TRAIN_EPOCH + TEST_EPOCH):
        input_target_sample = generateFn()
        input_color = [random.random() for color in range(INPUT_COLOR_DIMENSION)]
        correct_count = 0
        feed_stopped = False
        
        for t in range(LIGHT_EXPOSURE_TIME + WAITING_TIME):
            output_vector_prev = output_vector_list_prev.pop(0)
            input_env = [input_env[env] + 2*random.random()*INPUT_ENV_VOLATILITY - INPUT_ENV_VOLATILITY for env in range(INPUT_ENV_DIMENSION)]
            
            if t < LIGHT_EXPOSURE_TIME:
                input_vector = [*input_target_sample["input_color"], *input_env, *output_vector_prev]
            else:
                input_vector = [*[0 for color in range(INPUT_COLOR_DIMENSION)], *input_env, *output_vector_prev]
            
            target_vector = input_target_sample["target_vector"]
            
            a.propagateForward(input_vector)
            
            if a.lossFn(a.h[-1],[*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]]) < CORRECT_LOSS_LIMIT:
                if correct_count > LIGHT_EXPOSURE_TIME:
                    feed_stopped = True
                else:
                    correct_count += 1
                        
            if not feed_stopped and ep < TRAIN_EPOCH:
                a.propagateBackward([*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]])
                
            util.printAfterErase(f"iteration : {ep+1}; time : {t+1} ; loss : {a.lossFn(a.h[-1],[*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]])}     ")
            wr.writerow([ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,
                        ep+1,
                        target_vector[0]+2*target_vector[1], # none: 0, r: 1, b: 2
                        input_env[0],
                        t+1,
                        a.lossFn(a.h[-1],[*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]]),
                        1 if feed_stopped else 0,
                        correct_count
                        ])
            if SAVE_HEATMAP:
                a.saveHeatmap(f"output/{SAVE_HEATMAP_FILE_NAME}/{util.fillZero(ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,5)}.png")
            output_vector_list_prev.append(copy.deepcopy(a.h[-1]))

    f.close()
    print("\nDone")
    # clear_output()

convert output images into video